In [11]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
import mlflow


In [12]:
from mlflow.tracking import MlflowClient
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5001'
RUN_ID = 'b4d3bca8aa8e46a6b8257fe4541b1136'

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("green-taxi-duration")

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [13]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts


In [19]:
import pathlib
ec2_data_folder = pathlib.Path('/home/ubuntu/mlops_zoomcamp_sam/data')

df_train = read_dataframe(pathlib.Path(ec2_data_folder, 'green_tripdata_2021-01.parquet').resolve())
df_val = read_dataframe(pathlib.Path(ec2_data_folder, 'green_tripdata_2021-02.parquet').resolve())

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)
dict_location = 'dict_vectorizer.bin'
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)
    
    dv = DictVectorizer()
    model = RandomForestRegressor(**params, n_jobs=1)
    
    X_train = dv.fit_transform(dict_train)
    model.fit(X_train, y_train)
    
    X_val = dv.transform(dict_val)
    y_pred = model.predict(X_val)
    
    rmse = mean_squared_error(y_pred , y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)
    
    mlflow.sklearn.log_model(model, artifact_path='model')
    
    with open(dict_location, 'wb') as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact(dict_location)

mlflow.end_run()

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


FileNotFoundError: [Errno 2] No such file or directory: 'dict_vectorizor.bin'

In [16]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

/tmp/ipykernel_4564/1820300855.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')


RestException: RESOURCE_DOES_NOT_EXIST: Run with id=b4d3bca8aa8e46a6b8257fe4541b1136 not found